## SCEC Ground Motion Simulation Validation (GMSV) Toolkit FAS Workflow

In this notebook we present a brief introduction to SCEC GMSV Toolkit, and use it to compare the fourier amplitude spectra (FAS) of ground motion generated in the Broadband Platform (BBP) against recorded ground motions for the 1994 Northridge earthquake.

SCEC's goal with GMSV Toolkit is to make existing, well verified, and useful software available to a broad seismological and engineering community of researchers interested in ground motion simulations validation. SCEC's GMSV Toolkit leverages over a decade of scientific, engineering, and software development work completed by dozens of SCEC contributors, including the groups working with the Broadband Platform, the GMSV TAG, the High-F group, among others.

The GMSV codes compute various ground motion metrics, compare them with empirical data and models, and deliver suites of statistical products, including plots, to facilitate the evaluation. It aggregates methods into a standalone package so that their full capabilities are accessible for the validation of any simulation set.

### Overview

1. Getting started
2. Broadband Platform background
    1. Validation simulation inputs
    2. Validation simulation outputs
    3. Plotting map with stations and rupture
3. Calculating FAS
    1. Computing FAS for simulated and recorded data
4. Comparing FAS: simulations against observations
5. Generating FAS GoF Plot
    1. Calculating residuals
    2. FAS GoF
6. Further reading
7. Acknowledgements

### 1. Getting started

In this notebook we walk users through the steps involved in the GMSV Toolkit's FAS processing workflow. Several of the codes used in this notebook have been used in the post-processing stages of the Broadband Platform for more than ten years, and have been modified and migrated to the GMSV Toolkit repository for working as a standalone package. This enables users to take advantage of these codes outside of the Broadband Platform.

The codes included in the GMSV Toolkit package are designed to work independently and can be used separately. Each module includes a command-line interface that allows it to be used from the command-line or automated with a script. Additionally, users can import each module into their Python programs and use the tools directly by calling the functions they need. In this notebook we will focus on using the codes via the Python interface.

As a brief summary, in the FAS GoF workflow, acceleration timeseries go through the FAS module where fourier amplitude spectra is computed for frequencies between 0.01 and 100Hz. This output can be plotted using the plot_fas module and/or compared on a station by station basis with the plot_fas_comparison module. These FAS files can also be aggregated across all stations and compared against a second data set with the FAS GoF tool. Finally, the FAS GoF Plot module can be used to generate a FAS GoF plot so users can see how two datasets match. Please note that the workflow above can work with simulations versus observed data, as well as with two sets of simulated data.

The diagram below illustrates the processing steps involved in the FAS processing workflow:

<img align="center" width="800" src="images/gmsvtoolkit_fas_workflow.png">

We will start our processing with the outputs provided by the Platform, and use different tools from the GMSV Toolkit to obtain a FAS Goodness-of-Fit (GoF) plot that can be used to visualize how the simulation results from the Platform compare against recorded timeseries from the 1994 Northridge earthquake.

For this exercise, we will follow the FAS workflow described above. We will use the FAS module to compute fourier amplitude spectra for both calculated and observed timeseries. Then, we will generate comparison plots of observed versus calculated timeseries for a station to check how they compare at different frequencies. Next, we will combine the FAS data from all stations and generate residuals that will be used in the last step to generate a Goodness-of-Fit (GoF) plot.

Now, before we get started with this notebook, we will import a couple of basic modules that will provide useful functions for using paths and displaying plots later on.

In [ ]:
from IPython.display import Image, display
import os

### 2. Broadband Platform background

The Broadband Platform (BBP) is a software system that can generate 0-20+ Hz seismograms for historical and scenario earthquakes in California, Eastern North America, and Japan using several alternative computational methods. In validation simulations, we use a historical events and select station locations for which we have recorded data available. We can then compare the synthetic seismograms produced by the Platform against the recorded data and see how simulation methods match historical events. The Broadband Platform can also run scenario simulations, where it can generate ground motions for an arbitrary event at user-selected locations.

In this notebook, we will be using data from a Broadband Platform validation simulation that compares how a simulation method matches data recorded from the 1994 Northridge earthquake in Southern California.

#### A. Validation simulation inputs

<img align="right" width="350" src="images/labasin500.png">

Running the Broadband Platform involves selecting a simulation method and a simulation region that provides a 1-D velocity model that will be used to compute the synthetic seismograms. Although these items are not used in this notebook, we wanted to mention them for completeness. On the right, we show the 1-D velocity profile used for the LA Basin region, with a Vs30 set to 500m/s. After seismogram synthesis is completed, the Broadband Platform will run a site amplification module that will correct the simulated data at each station to the Vs30s provided in the station list (see below).

Also needed for a validation simulation is a description of the earthquake. The Broadband Platform uses a simple source description file (also called a SRC file with a ".src" extension). This file includes the fault geometry, location, mechanism, and other basic parameters used in the simulation. Users will also need to specify a list of stations for which we would like to compute synthetic seismograms. The station list contains information for each station, including station name, coordinates (latitude and longitude), Vs30, and also a range of frequencies for which recorded data is available.

Finally, for a validation simulation will need a set or recorded seismograms, one for each station, containing a three-component acceleration timeseries. These observations can then be compared against the simulated timeseries produced by the Platform using a variety of metrics.

Below we show examples of both a source description file and a station list, which will be used later in this notebook. The Northridge source description file includes magnitude, fault geometry and location, and other simulation parameters needed by the various Broadband Platform simulation methods. The station file contains the station location (longitude and latitude), a station name or identifier, Vs30 for the site, and the range of frequencies for which recorded data is valid for this station.

#### B. Validation simulation outputs

After the seismogram synthesis part of the Broadband Platform completes, we will have one velocity and one acceleration timeseries file for each station in our station list. These files, in BBP format, contain four columns, with time (in seconds), and two horizontal and one vertical components (in cm/s or cm/s/s). An example is provided below:

Please note that these are broadband seismograms, which combine both low- and high-frequency seismogram generation techniques from different Broadband Platform components. If the site response module was selected, these output timeseries have already been corrected to the Vs30 of each station in the station list.

#### C. Plotting map with stations and rupture

The data package provided with this notebook contains a full set of Broadband Platform seismograms for the 1994 Northridge earthquake. The set includes the Northridge source description file, a station list with 39 stations, and a set of observation files. We can set up paths for these files using the commands below:

In [ ]:
# First we configure the location of BBP input files
base_dir = "/home/scecuser/gmsvtoolkit_tutorials"
gmsv_data_package_dir = os.path.join(base_dir, "tutorial_data")
bbp_inputs = os.path.join(gmsv_data_package_dir, "bbp_inputs")

# Set up station list and source description files
station_file_3_stations = os.path.join(bbp_inputs, "nr_v19_06_2_3_stations.stl")
station_file_all_stations = os.path.join(bbp_inputs, "nr_v19_06_2.stl")
src_file = os.path.join(bbp_inputs, "nr_v20_07_1.src")

In [ ]:
# Set input folders for simulation and observation inputs

# These two folders also contain some preprocessed data to speed up
# some of the computations below, users can regenerate them using
# the tools available in this notebook
input_dir_sims = os.path.join(gmsv_data_package_dir, "bbp_results")
input_dir_obs = os.path.join(gmsv_data_package_dir, "obs_data")

# Set output folders where we want to store our results
output_dir_sims = os.path.join(gmsv_data_package_dir, "sims_processed")
output_dir_obs = os.path.join(gmsv_data_package_dir, "obs_processed")
output_dir = os.path.join(gmsv_data_package_dir, "gmsv_toolkit_fas_workflow")

In order to check the source description file and the station list, we will use the plot_map module in the plots package to create a map with the Northridge rupture and the list of stations used by the Broadband Platform. We will first import the plot_map module from the plots package:

In [ ]:
from plots.plot_map import plot_map

Next, we set a plot title, an output file, and invoke the plot_map method with the station list and source description file:

In [ ]:
plot_title = "Northridge Fault Trace with all Stations"
all_stations_map = "all_stations_map.png"
plot_map(station_file_all_stations, src_file, plot_title, output_dir, output_file=all_stations_map)

We can also generate a second map with a smaller set of stations that we will use for some of the examples below. We just substitute the station file and provide a different output file:

In [ ]:
plot_title = "Northridge Fault Trace with 3 Stations"
three_stations_map = "three_stations_map.png"
plot_map(station_file_3_stations, src_file, plot_title, output_dir, output_file=three_stations_map)

To visualize the two station maps we just generated, we can use the display function from Jupyter.

In [ ]:
for filename in [os.path.join(output_dir, all_stations_map), os.path.join(output_dir, three_stations_map)]:
    display(Image(filename))

### 3. Calculating FAS

For this step, we will compute FAS values for both simulated and observed data. We will be using GMSV Toolkit's FAS module, which can be used to read an acceleration timeseries and calculate FAS for a set of frequencies ranging from 0.01Hz to 100Hz.

We will import the FAS module from the metrics package and initialize the module:

In [ ]:
from metrics import fas
fas_obj = fas.FAS()

#### A. Computing FAS for simulated and recorded data

In this example we will compare FAS results from two sets of data: simulated seismograms produced by the Broadband Platform, and observed data recorded at each site. The required inputs for the FAS module include a list of stations and input acceleration timeseries for each set of stations. For this example, we will use a station list with only three stations. The FAS module expects timeseries for each set of stations to be in a single folder, and that all input files include the station name and by default feature a ".acc.bbp" extension. Additionally, we will provide an output folder where we would like to store the computed FAS for each station.

With all input files in place, the next step is to run the FAS code, which will compute fourier amplitude spectra for each of the stations included in the station list. Differently from most modules in the GMSV Toolkit software, the FAS module requires all stations and data sets that will be compared to be specified at once. This allows the code to resample all timeseries to a common DT and to prepare them for comparison.

In [ ]:
# Generate a list of folders where the data will be coming from, also specify labels for each data set
input_fas_dirs = [input_dir_sims, input_dir_obs]
labels_fas = ["2354660", "obs"]

# Now we can run the FAS module
fas_obj.run_validation(station_file_3_stations, input_fas_dirs, labels_fas, output_dir)

### 4. Comparing FAS: simulations against observations

Now that we have FAS files for both simulated and observed data, we can create a plot comparing them using the plot_fas_comparison module. We import the plot_fas_comparison module from the plots package and initialize it:

In [ ]:
from plots import plot_fas_comparison

We are going to use the same station list and we just create an "input_dirs" array containing a list of folders where our data sets can be found. Additionally, we create a "labels" array containing labels for each of the data sets we specified in the "input_dirs" array. For our example, we will have two labels, "obs" refers to the observed data from the Northridge event, and we will use the Broadband Platform simulation id (2354660) for the simulated data. Setting up comp_label is optional, and allows us to specify a prefix to the output png file. 

In [ ]:
input_dirs = [output_dir, output_dir]
labels = ["obs", "2354660"]
comp_label = "NR_2354660"

Once we have everything set up, we can just call the run_station_mode method and we should get a comparison plot for each station in our station list.

In [ ]:
plot_fas_comparison.run_station_mode(station_file_3_stations, input_dirs, labels, output_dir, comp_label)

Finally, we can see the plot comparison for station 2001-SCE using the following command:

In [ ]:
display(Image(os.path.join(output_dir, "NR_2354660.2001-SCE.fas.comparison.png")))

In the plots above we can see FAS values for the two horizontal components and Smoothed EAS (right). The black line shows calculated data and the red line shows FAS values for the recorded seismogram. The two vertical lines (purple and red) come from the station list and indicate limits for which the recorded data is valid for this particular station.

### 5. Generating FAS GoF plot

Now that we have computed FAS for both simulations and observed data, we can generate a Goodness-of-Fit (GoF) plot using the two datasets. Although we can generate a GoF plot with the 3 stations that we have processed above, it will look better with more stations so that's what we will use. The training data package includes pre-computed FAS files for all stations (both simulated and observed data), but users are welcome to replace the short station list with the full station list in the examples above and regenerate the files themselves, it will just take a little longer.

#### A. Calculating residuals

First we import the fas_gof module from the stats package. We set the comp_label parameter to "NR_2354660", signaling we are comparing recorded data from the Northridge earthquake against the Broadband simulation 2354660. This will be used to generate a file prefix for the various residual files that will be produced by the fas_gof module. We can also use the max_cutoff parameter to exclude stations that are too far away from the rupture. In our example, we set it to 120km to include all stations in our list:

In [ ]:
from stats import fas_gof
max_cutoff = 120
comp_label = "NR_2354660"
method = "gp"

Next, we instantiate our FASGoF module and run it to generate the residual files for all stations:

In [ ]:
fas_gof_obj = fas_gof.FASGoF(max_cutoff=max_cutoff, comp_label=comp_label)
fas_gof_obj.run_fas_gof(station_file_all_stations, src_file, input_dir_obs, input_dir_sims, output_dir)

#### B.  Plotting GoF

Now that we have calculated the residuals using the fas_gof module, we can generate a GoF plot using the plot_fas_gof method available in the plots package. We import plot_fas_gof, set up a plot_title for our plot, and generate the FAS GoF using the following code:

In [ ]:
from plots.plot_fas_gof import plot_fas_gof
plot_title = "GoF Comparison between NR and simulation 2354660"
plot_fas_gof(plot_title, comp_label, output_dir, output_dir, max_cutoff=max_cutoff, method=method)

Please note that the optional method parameter above is used to add vertical lines to the plot that indicate the frequency range where simulated data for this method is valid. Users also have the option to specify the frequency range themselves by using the optional lfreq and hfreq parameters.

Finally, we can visualize the generated GoF plot using the command below:

In [ ]:
display(Image(os.path.join(output_dir, "gof-NR_2354660_r0-120-fas.png")))

For the plot above we will focus on the first subplot, which features Smoothed EAS values. This plot includes data from all stations and shows how simulated data compares against recorded data at different frequencies. The solid red line shows the mean, the narrow band is the 90% confidence interval of the mean, and the wide band shows the standard deviation centered around the mean. Note that for frequencies between 0.15Hz and 1Hz the red line is closer to zero, indicating the two sets are close. For frequencies > 1.0Hz the red line dips below zero, signaling the simulated data is overpredicting the recorded data.

### 6. Further reading

#### Software reference

The primary reference for the Broadband Platform software system (v15.3.0 and later) is:

* Maechling, P. J., F. Silva, S. Callaghan, and T. H. Jordan (2015). SCEC Broadband Platform: System Architecture and Software Implementation, Seismol. Res. Lett., 86, no. 1, doi: 10.1785/0220140125

#### Ground Motion Simulation Validation Method Reference
The primary reference that describes the validation process developed by the Broadband Platform group to establish that the BBP platform software produces results are suitable for use in engineering applications is:

* Goulet, C.A., Abrahamson, N.A., Somerville, P.G. and K, E. Wooddell (2015) The SCEC Broadband Platform Validation Exercise: Methodology for Code Validation in the Context of Seismic-Hazard Analyses, Seismol. Res. Lett., 86, no. 1, doi: 10.1785/0220140104

#### GitHub repo

Software related to this notebook can be found on GitHub:

* GMSV Toolkit: https://github.com/SCECcode/gmsvtoolkit

* GMSV Toolkit Tutorial: https://github.com/SCECcode/gmsvtoolkit_tutorials

* SCEC Broadband Platform: https://github.com/SCECcode/bbp

### 7. Acknowledgements

The following developers have contributed to the development of the GMSV Toolkit software package.

* Norm Abrahamson, Pacific Gas & Electric (PG&E)
* John Anderson, University of Nevada, Reno (UNR)
* Ralph Archuleta, University of California Santa Barbara (UCSB)
* Domniki Asimaki, California Institute of Technology (Caltech)
* Karen Assatourians, Western University, Canada
* Gail Atkinson, Western University, Canada
* Shima Azizzadeh-Roodpish, University of Memphis
* Jeff Bayless, AECOM
* Alexander Nikolas Breuer, Friedrich Schiller University Jena
* Scott Callaghan, Southern California Earthquake Center (SCEC)
* Jorge Crempien, Pontificia Universidad Católica de Chile
* Christine Goulet, Southern California Earthquake Center (SCEC)
* Robert Graves, United States Geological Survey (USGS)
* Behzad Hassani, British Columbia Hydro and Power Authority (BC Hydro)
* Asako Iwaki, National Research Institute for Earth Science and Disaster Resilience (NIED)
* Chen Ji, University of California Santa Barbara (UCSB)
* Thomas H. Jordan, Southern California Earthquake Center (SCEC)
* Naeem Khoshnevis, University of Memphis
* Philip Maechling, Southern California Earthquake Center (SCEC)
* Hiroe Miyake, Earthquake Research Institute, University of Tokyo
* Kim Olsen, San Diego State University (SDSU)
* Arben Pitarka, Lawrence Livermore National Laboratory (LLNL)
* Sanaz Rezaeian, United States Geological Survey (USGS)
* William Savran, San Diego State University (SDSU)
* Jian Shi, California Institute of Technology (Caltech)
* Fabio Silva, Southern California Earthquake Center (SCEC)
* Andreas Skarlatoudis, AECOM
* Patrick Small, Southern California Earthquake Center (SCEC)
* Paul Somerville, AECOM
* Seok-Goo Song, Korea Institute of Geoscience and Mineral Resources (KIGAM)
* Ricardo Taborda, Universidad EAFIT
* Rumi Takedatsu, San Diego State University (SDSU)
* Nan Wang, San Diego State University (SDSU)
* Ke Xu, San Diego State University (SDSU)